In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
DATA_FILE = './birth_life_2010.txt'

In [ ]:
batch_size = 10
n_epoch = 100

In [ ]:
def read_birth_life_data(filename):
    """
    Read in birth_life_2010.txt and return:
    data in the form of NumPy array
    n_samples: number of samples
    """
    text = open(filename, 'r').readlines()[1:]
    data = [line[:-1].split('\t') for line in text]
    births = [float(line[1]) for line in data]
    lifes = [float(line[2]) for line in data]
    data = list(zip(births, lifes))
    n_samples = len(data)
    data = np.asarray(data, dtype=np.float32)
    return data, n_samples

In [ ]:
# Step 1: read in data from the .txt file
data, n_samples = read_birth_life_data(DATA_FILE)

In [ ]:
# Step 2: create placeholders for X (birth rate) and Y (life expectancy)
X = tf.placeholder(tf.float32, [None,], name='X')
Y = tf.placeholder(tf.float32, [None,], name='Y')

In [ ]:
# Step 3: create weight and bias, initialized to 0
w = tf.get_variable('weights', initializer=tf.constant(0.0))
b = tf.get_variable('bias', initializer=tf.constant(0.0))

In [ ]:
# Step 4: build model to predict Y
hypothesis = w * X + b 

In [ ]:
# Step 5: use the squared error as the loss function
# you can use either mean squared error or Huber loss
loss = tf.reduce_mean(tf.square(Y - hypothesis), name='loss')

In [ ]:
# Step 6: using gradient descent with learning rate of 0.001 to minimize loss
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

In [ ]:
writer = tf.summary.FileWriter('./graphs/linear_reg_minibatch', tf.get_default_graph())
with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True))) as sess:
    # Step 7: initialize the necessary variables, in this case, w and b
    sess.run(tf.global_variables_initializer())
    
    total_batch = int(n_samples/batch_size)
    
    # Step 8: train the model for 100 epochs
    for epoch in range(n_epoch): 
        total_loss = 0
        for i in range(total_batch):
            x_data = data[i*batch_size:(i+1)*batch_size, 0]
            y_data = data[i*batch_size:(i+1)*batch_size, 1]
            # Session execute optimizer and fetch values of loss
            _, l = sess.run([optimizer, loss], feed_dict={X: x_data, Y:y_data}) 
            total_loss += l
        print('Epoch {0}: {1}'.format(epoch, total_loss/total_batch))

    # close the writer when you're done using it
    writer.close() 
    
    # Step 9: output the values of w and b
    w_out, b_out = sess.run([w, b])

In [ ]:
# plot the results
plt.plot(data[:,0], data[:,1], 'bo', label='Real data')
plt.plot(data[:,0], data[:,0] * w_out + b_out, 'r', label='Predicted data')
plt.legend()
plt.show()